In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from progressBar import printProgressBar

import medicalDataLoader
import argparse
from utils import *

from UNet_Base import *
import random
import torch
import pdb

In [2]:
import warnings
warnings.filterwarnings("ignore")

**Format [B, C, H, W]** :

- B	Taille du batch
- C	Nombre de classes 
- H	Hauteur de l’image
- W	Largeur 

--> pred est au format [B, C, H, W] ie :
- Pour **B** images, pour i allant de 0 à B --> pred[image i] contient **C** matrices (= une matrice par classe) de dimension **H*W**, avec un logit (=output avant le softmax) pour chaque pixel.

In [ ]:
def dice_score(pred, target, num_classes, eps=1e-7):
    """
    pred: voir le markdown plus haut   
    target: format [B, H, W] --> contient une seule valeur (et une seule matrice) par pixel : 
            (0=background, 1=classe 1, 2=classe 2, 3=classe 3)
    """
    pred_classes = torch.argmax(pred, dim=1)  # retourne une pred par pixel au format [B, H, W]

    dice_per_class = []
    for c in range(num_classes):
        pred_c = (pred_classes == c).float()
        target_c = (target == c).float()

        intersection = (pred_c * target_c).sum()
        union = pred_c.sum() + target_c.sum()

        dice = (2 * intersection + eps) / (union + eps)
        dice_per_class.append(dice)

    # retoune un tensor shape [num_classes]
    return torch.stack(dice_per_class)

In [4]:
def runTraining():
    print('-' * 40)
    print('~~~~~~~~  Starting the training... ~~~~~~')
    print('-' * 40)

    ## DEFINE HYPERPARAMETERS (batch_size > 1)
    batch_size = 4
    batch_size_val = 4 
    lr = 0.01    # Learning Rate
    epoch = 10 # Number of epochs
    
    root_dir = './Data/'

    print(' Dataset: {} '.format(root_dir))

    ## DEFINE THE TRANSFORMATIONS TO DO AND THE VARIABLES FOR TRAINING AND VALIDATION
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    mask_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    train_set_full = medicalDataLoader.MedicalImageDataset('train',
                                                      root_dir,
                                                      transform=transform,
                                                      mask_transform=mask_transform,
                                                      augment=False,
                                                      equalize=False)

    train_loader_full = DataLoader(train_set_full,
                              batch_size=batch_size,
                              worker_init_fn=np.random.seed(0),
                              num_workers=0,
                              shuffle=True)


    val_set = medicalDataLoader.MedicalImageDataset('val',
                                                    root_dir,
                                                    transform=transform,
                                                    mask_transform=mask_transform,
                                                    equalize=False)

    val_loader = DataLoader(val_set,
                            batch_size=batch_size_val,
                            worker_init_fn=np.random.seed(0),
                            num_workers=0,
                            shuffle=False)

    # VALIDATION FUNCTION
    def validate(model, loader, criterion, num_classes):
        model.eval()
        total_loss = 0.0
        dice_total = []

        with torch.no_grad():
            for images, labels, _ in loader:
                images = to_var(images)
                labels = to_var(labels)

                outputs = model(images)
                segmentation_classes = getTargetSegmentation(labels)

                loss = criterion(outputs, segmentation_classes)
                total_loss += loss.item()

                dice = dice_score(outputs, segmentation_classes, num_classes)
                dice_total.append(dice.cpu().numpy())

        dice_mean = np.mean(np.array(dice_total), axis=0)  # dice pour chaque classe
        return total_loss / len(loader), dice_mean



    ## INITIALIZE YOUR MODEL
    num_classes = 4 # NUMBER OF CLASSES

    print("~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~")
    modelName = 'Test_Model'
    print(" Model Name: {}".format(modelName))

    ## CREATION OF YOUR MODEL
    net = UNet(num_classes)

    print("Total params: {0:,}".format(sum(p.numel() for p in net.parameters() if p.requires_grad)))

    # DEFINE YOUR OUTPUT COMPONENTS (e.g., SOFTMAX, LOSS FUNCTION, ETC)
    softMax = torch.nn.Softmax(dim=1)
    CE_loss = torch.nn.CrossEntropyLoss()

    ## PUT EVERYTHING IN GPU RESOURCES    
    if torch.cuda.is_available():
        net.cuda()
        softMax.cuda()
        CE_loss.cuda()

    ## DEFINE YOUR OPTIMIZER
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    ### To save statistics ####
    lossTotalTraining = []
    Best_loss_val = 1000
    BestEpoch = 0
    
    directory = 'Results/Statistics/' + modelName

    print("~~~~~~~~~~~ Starting the training ~~~~~~~~~~")
    if os.path.exists(directory)==False:
        os.makedirs(directory)

    ## START THE TRAINING
    
    ## FOR EACH EPOCH
    for i in range(epoch):
        net.train()
        lossEpoch = []
        DSCEpoch = []
        DSCEpoch_w = []
        num_batches = len(train_loader_full)
        
        ## FOR EACH BATCH
        for j, data in enumerate(train_loader_full):
            ### Set to zero all the gradients
            net.zero_grad()
            optimizer.zero_grad()

            ## GET IMAGES, LABELS and IMG NAMES
            images, labels, img_names = data

            ### From numpy to torch variables
            labels = to_var(labels)
            images = to_var(images)

            ################### Train ###################
            #-- The CNN makes its predictions (forward pass)
            net_predictions = net(images)

            #-- Compute the losses --#
            # THIS FUNCTION IS TO CONVERT LABELS TO A FORMAT TO BE USED IN THIS CODE
            segmentation_classes = getTargetSegmentation(labels)
            
            # COMPUTE THE LOSS
            CE_loss_value = CE_loss(net_predictions, segmentation_classes) # XXXXXX and YYYYYYY are your inputs for the CE
            lossTotal = CE_loss_value

            # DO THE STEPS FOR BACKPROP (two things to be done in pytorch)
            
            # THIS IS JUST TO VISUALIZE THE TRAINING 
            lossEpoch.append(lossTotal.cpu().data.numpy())
            printProgressBar(j + 1, num_batches,
                             prefix="[Training] Epoch: {} ".format(i),
                             length=15,
                             suffix=" Loss: {:.4f}, ".format(lossTotal))

        lossEpoch = np.asarray(lossEpoch)
        lossEpoch = lossEpoch.mean()

        lossTotalTraining.append(lossEpoch)

        # ---- DICE SCORE ---- #
        with torch.no_grad():
            dice = dice_score(net_predictions, segmentation_classes, num_classes)
            DSCEpoch.append(dice.cpu().numpy())


        printProgressBar(num_batches, num_batches,
                             done="[Training] Epoch: {}, LossG: {:.4f}".format(i,lossEpoch))
        
        # VALIDATION PHASE 
        val_loss, val_dice = validate(net, val_loader, CE_loss, num_classes)
        print(f"[Validation] Epoch: {i}, Loss: {val_loss:.4f}, Dice: {val_dice}")

        # SAVE BEST MODEL
        if val_loss < Best_loss_val:
            Best_loss_val = val_loss
            BestEpoch = i

            if not os.path.exists('./models/' + modelName):
                os.makedirs('./models/' + modelName)

            torch.save(net.state_dict(), './models/' + modelName + '/' + str(i) + '_Epoch')
            print(f"Best model updated at epoch {i} (val_loss={val_loss:.4f})")

    print("\n----------------------------------------")
    print(f"Best model found at epoch {BestEpoch} with validation loss = {Best_loss_val:.4f}")
    print("----------------------------------------\n")


        # ## THIS IS HOW YOU WILL SAVE THE TRAINED MODELS AFTER EACH EPOCH. 
        # ## WARNING!!!!! YOU DON'T WANT TO SAVE IT AT EACH EPOCH, BUT ONLY WHEN THE MODEL WORKS BEST ON THE VALIDATION SET!!
        # if not os.path.exists('./models/' + modelName):
        #         os.makedirs('./models/' + modelName)

        #     torch.save(net.state_dict(), './models/' + modelName + '/' + str(i) + '_Epoch')
            
        # np.save(os.path.join(directory, 'Losses.npy'), lossTotalTraining)


À chaque epoch, le modèle passe successivement par :

- une phase d'entraînement (LossG),
- une phase de validation (val_loss),
- une vérification si le modèle est le meilleur jusqu’ici.

In [5]:
runTraining()

----------------------------------------
~~~~~~~~  Starting the training... ~~~~~~
----------------------------------------
 Dataset: ./Data/ 
~~~~~~~~~~~ Creating the UNet model ~~~~~~~~~~
 Model Name: Test_Model
Total params: 60,664
~~~~~~~~~~~ Starting the training ~~~~~~~~~~
[Training] Epoch: 0 [DONE]                                 
[Training] Epoch: 0, LossG: 1.8079                                                                           
[Validation] Epoch: 0, Loss: 1.6403, Dice: [0.25092205 0.00207371 0.0196559  0.014097  ]
Best model updated at epoch 0 (val_loss=1.6403)
[Training] Epoch: 1 [DONE]                                 
[Training] Epoch: 1, LossG: 1.8074                                                                           
[Validation] Epoch: 1, Loss: 1.7036, Dice: [0.3067482  0.00233595 0.01749299 0.01859846]
[Training] Epoch: 2 [DONE]                                 
[Training] Epoch: 2, LossG: 1.8084                                                            

On observe que la perte d’entraînement reste stable autour de 1.07 pour toutes les époques :
- Cela indique que le modèle baseline apprend très peu, ou pas du tout.
- Le modèle apprend un peu au début (jusqu'à epoch : 6), puis commence à stagner ou légèrement sur-apprendre
- Meilleur modèle = epoch 6

A faire ensuite :
- Améliore le modèle
- faire un dice score
- visualiser les différentes étapes/prédictions

**Selon l'ami le chat :**

🔍 Que vérifier maintenant ?

Parce que ton loss stable est louche, tu devrais vérifier :

✔ 1. getTargetSegmentation(labels)

Est-ce que ça renvoie bien un tenseur [B,H,W] d'entiers (classes) ?

✔ 2. Les labels contiennent plusieurs classes ?

Si tout est 0 → CE ~1.07 est normal.

✔ 3. Le learning rate

lr = 0.01 est assez agressif pour UNet.
Essaie 0.001 ou 0.0001.

✔ 4. Visualiser quelques prédictions

Pour vérifier que les masques prévus ne sont pas tous pareils.